In [15]:
import pandas as pd
import json
import requests
import time
import threading
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta


url = "https://login-p10.xiaoshouyi.com/auc/oauth2/token"
payload = {
    "grant_type": "password",
    "client_id": "351fb11916fc85504ba5f6ec3b96988b",
    "client_secret": "8f9355878504005ba782ca049d013d4e",
    "redirect_uri": "https://crm-p10.xiaoshouyi.com",
    "username": "11020964@twkd.com",
    "password": "Kd0627ubGHagpQ"
}

response = requests.post(url, data=payload)
content = response.json()
ac_token = content["access_token"]

date_1 = datetime.now() - timedelta(days=3)
date_1_scrm = int(date_1.timestamp() * 1000)
date_0 = datetime.now()
date_0_scrm = int(date_0.timestamp()*1000)

'''
obtain table type to get the entityType
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/xobjects/customEntity47__c/busiType"
headers = {
    "Authorization": f"Bearer {ac_token}"
    #"Content-Type":"application/json"    #json data type
}
response2 = requests.get(url_2, headers = headers)
content2 = response2.json()
print(content2)
entityType_df = pd.json_normalize(content2["data"]["records"])
# Daily Invite Mr.K Call = 3093637455518085
# 新客電訪交辦 = 3126024717160324
'''
select from user
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
data = {
    "xoql": "select id, name from user",
    #"xoql": "insert into contact(contactName, phone, email) values('貴夫人', '0912345678', 'kkdd@gmail.com')"
    "batchCount": 2000
}
response2 = requests.post(url_2, headers=headers, data = data)
ownerId = response2.json()
print(ownerId)
user_df = pd.json_normalize(ownerId["data"]["records"])

'''
select dimdepart
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
data = {
    "xoql": "select id, departCode, departName, departType, erSerivce__c from department",
    #"xoql": "insert into contact(contactName, phone, email) values('貴夫人', '0912345678', 'kkdd@gmail.com')"
    "batchCount": 2000
}
response2 = requests.post(url_2, headers=headers, data = data)
dimdepart = response2.json()
print(dimdepart)
filtered_data = []
for app in dimdepart["data"]['records']:
    app_data = {
        "departName": app["departName"],
        "id": app["id"]
    }
    filtered_data.append(app_data)
dimdepart_df = pd.DataFrame(filtered_data)
dimdepart_df = dimdepart_df.rename(columns={'departName': 'DataRegionGroupID', 'id':'dimDepart'})
# chen = user_df[user_df['name']=='陳信榮']
# print(chen)
# (entityType = '3093637455518085' or entityType = '3126024717160324') 
'''
select from Tasks withdraw (customItem39__c 是否邀約K大 , customItem38__c 電訪類型)
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
Tasks_df = pd.DataFrame()

while True:
    data = {
        "xoql": f'''
        select id,name,approvalStatus, customItem22__c as 執行狀態 ,entityType
        from customEntity47__c
        where (entityType = '3093637455518085' or entityType = '3126024717160324') 
        and customItem39__c is null and customItem38__c is null 
        and createdBy = '2988448346347872'
        and customItem7__c <={date_0_scrm}
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Tasks_df = pd.concat([Tasks_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
#createdAt >= {date_3_scrm}
#customItem7__c <={date_0_scrm}
    
Tasks_df['執行狀態']= Tasks_df['執行狀態'].astype(str)
Tasks_df1 = Tasks_df.loc[Tasks_df['執行狀態'].str.contains("等待")]  
Tasks_df1['approvalStatus']= Tasks_df1['approvalStatus'].astype(str)
Tasks_df1 = Tasks_df1.loc[Tasks_df1['approvalStatus'].str.contains("撤回")]
#Tasks_df1 = Tasks_df1.iloc[:4000]


{'code': '200', 'msg': None, 'data': {'totalSize': 9, 'count': 9, 'records': [{'id': 3093637455518085, 'apiKey': 'defaultBusiType', 'label': 'Daily Invite Mr.K Call', 'labelKey': 'XdMDBizTp.customEntity47__c.defaultBusiType', 'helpText': '', 'helpTextKey': 'XdMDBizTp.Hlptxt.customEntity47__c.defaultBusiType', 'description': '', 'active': True, 'custom': False}, {'id': 3116404355485607, 'apiKey': 'businessType2__c', 'label': 'After Sending Catalog Call', 'labelKey': 'XdMDBizTp.customEntity47__c.businessType2__c', 'helpText': '', 'helpTextKey': 'XdMDBizTp.Hlptxt.customEntity47__c.businessType2__c', 'description': '', 'active': True, 'custom': True}, {'id': 3122959319912325, 'apiKey': 'businessType3__c', 'label': '電/拜訪交辦管理', 'labelKey': 'XdMDBizTp.customEntity47__c.businessType3__c', 'helpText': '', 'helpTextKey': 'XdMDBizTp.Hlptxt.customEntity47__c.businessType3__c', 'description': '', 'active': True, 'custom': True}, {'id': 3124313942183820, 'apiKey': 'businessType4__c', 'label': '一般交辦'

C:\Users\11020964\AppData\Local\Temp\ipykernel_21036\43035037.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tasks_df1['approvalStatus']= Tasks_df1['approvalStatus'].astype(str)


In [16]:
Tasks_df1

,id,name,approvalStatus,執行狀態,entityType
1,3611146377910602,GTO02201423,['撤回'],['等待回應'],3126024717160324
2,3611146378795347,GTO02201066,['撤回'],['等待回應'],3126024717160324
3,3611146380562376,GTO02201601,['撤回'],['等待回應'],3126024717160324
4,3629395941005673,GTO02265960,['撤回'],['等待回應'],3126024717160324
6,3720172510402920,GTO02604101,['撤回'],['等待回應'],3126024717160324
...,...,...,...,...,...
10471,3730460995751879,GTO02631305,['撤回'],['等待回應'],3093637455518085
10472,3730460995751880,GTO02631306,['撤回'],['等待回應'],3093637455518085
10473,3730460995751882,GTO02631308,['撤回'],['等待回應'],3093637455518085
10474,3730460995751886,GTO02631312,['撤回'],['等待回應'],3093637455518085


In [17]:
'''
get procInstId
'''
Tasks_df2 = Tasks_df1[['id']]
base_url = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/creekflow/history/filter?entityApiKey=customEntity47__c&dataId="

def fetch_data(data_id):
    #print(data_id)
    api_url = f"{base_url}{data_id}&stageFlg=false"
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        api_data = response.json()
        if api_data["data"]:
          return json.dumps({ "dataId": data_id, "procInstId": api_data["data"][-1]["procInstId"] })
          # df = pd.DataFrame(api_data["data"][0]["procInstId"], dtype=str)
        else:
            #print("Data is empty")
            return
    else:
        #print(f"Error accessing API for dataId {data_id}. Status code: {response.status_code}")
        # return pd.DataFrame()
        return
    

# Set the maximum number of threads you want to use
max_threads = 2  # You can adjust this based on your needs
with ThreadPoolExecutor(max_threads) as executor:
    # Use executor.map to asynchronously fetch data for each row in parallel
    dfs = list(executor.map(fetch_data, Tasks_df2['id']))
    # Add a time delay between each thread

json_data = [entry for entry in dfs if entry is not None]
Tasks_df2 = pd.json_normalize([json.loads(entry) for entry in json_data])




KeyboardInterrupt: 

In [ ]:
'''
withdraw_task
'''
def withdraw_task(row):
    data_id = row['dataId']
    task_id = row['procInstId']
    url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/creekflow/task"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type": "application/json"
    }
    data = {
        "data": {
            "action": "withdraw",
            "entityApiKey": "customEntity47__c",
            "dataId": data_id,
            "procInstId": task_id
        }
    }

    response = requests.post(url_2, headers=headers, json=data)
    result = response.json()
    print(f"Response for dataId {data_id}: {result}")

# Assuming data_ids is your DataFrame
data_ids_df = Tasks_df2[['dataId', 'procInstId']]

# Create a thread for each row in the DataFrame
threads = []
for index, row in data_ids_df.iterrows():
    thread = threading.Thread(target=withdraw_task, args=(row,))
    threads.append(thread)
    thread.start()
    time.sleep(0.08)

# Wait for all threads to finish
for thread in threads:
    thread.join()


In [18]:
'''
ask bulk id
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/bulk/v2/job"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/json"
}
data = {
     "data": {
        "operation": "delete",####################################################################3
        "object": "customEntity47__c",
        "execOption": ["CHECK_RULE", "CHECK_DUPLICATE"]
    }
    
}
response = requests.post(url_2, headers = headers, json = data)
test = response.json()
bulk_id = test["result"]["id"]
print(bulk_id)

Tasks_df3 = Tasks_df2[['dataId']]
Tasks_df3 = Tasks_df3.rename(columns={'dataId':'id'})
#Tasks_df3 = Tasks_df1[['id']]

KeyError: 'id'